In [ ]:
pip install bs4

In [ ]:
pip install html5lib

In [ ]:
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

In [ ]:
URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1"

In [ ]:

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
r = requests.get(url=URL, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

In [ ]:
players_list = []
age_list = []
position_list = []
value_list = []
nationality_list = []

In [ ]:
players = soup.find_all("td", class_="hauptlink")
    

In [ ]:
players

In [ ]:
for player in players:
    players_list.append(player.text)

In [ ]:
players_list

In [ ]:

i = 1
while i < len(players_list):
    del players_list[i]
    i += 1
print(players_list)


In [ ]:
ages = soup.find_all("td", class_="zentriert")

In [ ]:
ages

In [ ]:
for age in ages:
    age_list.append(age.text)

In [ ]:
age_list

In [ ]:
new_age_list = []
for i in range(2, len(age_list), 4):
    new_age_list.append(age_list[i])
print(new_age_list)

In [ ]:
new_age_list

In [ ]:
nationality = soup.find_all("td", class_="zentriert")

In [ ]:
nationality

In [ ]:

for nation in nationality:
    flags = nation.find_all('img')  
    if flags:  
        first_flag = flags[0]  
        first_nationality = first_flag['title']
        nationality_list.append(first_nationality)
        print(first_nationality)


In [ ]:
i = 1
while i < len(nationality_list):
    del nationality_list[i]
    i += 1
print(nationality_list)

In [266]:
values = soup.find_all("td", class_="rechts hauptlink")

In [267]:
values

[<td class="rechts hauptlink"><a href="/erling-haaland/marktwertverlauf/spieler/418560">€170.00m</a> </td>,
 <td class="rechts hauptlink"><a href="/phil-foden/marktwertverlauf/spieler/406635">€110.00m</a> </td>,
 <td class="rechts hauptlink"><a href="/bukayo-saka/marktwertverlauf/spieler/433177">€110.00m</a> <span class="icons_sprite green-arrow-ten" title="Previous market value: €100.00m"> </span></td>,
 <td class="rechts hauptlink"><a href="/harry-kane/marktwertverlauf/spieler/132098">€90.00m</a> </td>,
 <td class="rechts hauptlink"><a href="/enzo-fernandez/marktwertverlauf/spieler/648195">€85.00m</a> <span class="icons_sprite green-arrow-ten" title="Previous market value: €55.00m"> </span></td>,
 <td class="rechts hauptlink"><a href="/declan-rice/marktwertverlauf/spieler/357662">€80.00m</a> </td>,
 <td class="rechts hauptlink"><a href="/rodri/marktwertverlauf/spieler/357565">€80.00m</a> </td>,
 <td class="rechts hauptlink"><a href="/martin-odegaard/marktwertverlauf/spieler/316264">€

In [268]:
for value in values:
    value_list.append(value.text)

In [269]:
value_list

['€170.00m\xa0',
 '€110.00m\xa0',
 '€110.00m\xa0\xa0',
 '€90.00m\xa0',
 '€85.00m\xa0\xa0',
 '€80.00m\xa0',
 '€80.00m\xa0',
 '€80.00m\xa0\xa0',
 '€80.00m\xa0',
 '€80.00m\xa0',
 '€80.00m\xa0\xa0',
 '€75.00m\xa0',
 '€75.00m\xa0',
 '€75.00m\xa0',
 '€75.00m\xa0',
 '€70.00m\xa0\xa0',
 '€70.00m\xa0',
 '€70.00m\xa0',
 '€70.00m\xa0',
 '€70.00m\xa0\xa0',
 '€70.00m\xa0\xa0',
 '€65.00m\xa0',
 '€65.00m\xa0\xa0',
 '€65.00m\xa0\xa0',
 '€60.00m\xa0',
 '€110.00m\xa0\xa0',
 '€85.00m\xa0\xa0',
 '€80.00m\xa0\xa0',
 '€80.00m\xa0\xa0',
 '€70.00m\xa0\xa0']